In [1]:
import pandas as pd
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel

quetzal_path = r'../../model/'
input_path = r'../../inputs/'

sm = stepmodel.read_zip(quetzal_path + 'preparation/networkcasted.zip')
pool = sm.copy()

SQLalchemy is not installed. No support for SQL output.


zones: 100%|██████████| 27/27 [00:00<00:00, 47.90it/s]       


# colors

In [2]:
rainbow_shades = ["#D22328", "#559BB4", "#91A564", "#DC9100", "#8C4B7D", "#A08C69",
                  "#647D6E", "#5A7382", "#64411E", "#A00037", "#643C5A"]

#sm.links['trip_id'] = sm.links['dir_line']
sm.analysis_lines()
sm.lines['line_color'] = list(rainbow_shades[:5]*10000)[: len(sm.lines)]
d = sm.lines.set_index('route_id')['line_color'].to_dict()
sm.links['line_color'] = sm.links['route_id'].apply(
    lambda x: d[x] if x in d.keys() else "#D22328"
)

In [3]:
parameter_frame = pd.read_excel(input_path + r'parametros.xlsx').set_index(['category','parameter'])

# parameters

In [4]:
for c in parameter_frame.columns:
    parent = parameter_frame[c][('general', 'parent')]
    try:
        parent = parent.iloc[0]
    except AttributeError:# 'str' object has no attribute 'iloc'
        pass
    parameter_frame[c] = parameter_frame[c].fillna(parameter_frame[parent])

In [5]:
gtfs = stepmodel.read_zip(r'../../model/preparation/gtfs.zip', only_attributes=['links', 'jsons'])
networkcasted_routes = set(gtfs.links['route_id'])
parameter_routes = set(parameter_frame.loc['route_type'].index)
drop_routes = networkcasted_routes - parameter_routes
if len(drop_routes):
    print('routes that belong to gtfs and not to parameters (will not be used):')
    print(drop_routes)
pool.links = pool.links.loc[~pool.links['route_id'].isin(drop_routes)]

nodes: 100%|██████████| 3/3 [00:00<00:00, 20.87it/s]

routes that belong to gtfs and not to parameters (will not be used):
{'tramo SDO L2', 'quentin', 'sitp_tiradentes', 'Tele Oeste SDO', 'intrant', 'Tramo Alcarrizos L2', 'Tele Oeste LA'}


In [6]:
links = sm.links
print(set(links.loc[links['route_type'] == 'bus']['route_id']))

{'sitp_lincoln', 'independencia', 'AB_A20B_1', 'sitp_prolongacion_27', 'sitp_josefa', 'AB_A20A_2', 'sitp_churchill', '27_febrero', 'AB_A11_2', 'AB_A69_1', 'AB_A20B_2', 'AB_A11_1', 'AB_A-B_1', 'AB_A69_2', 'sitp_defillo', 'AB_A31_1', 'sitp_cdg_oeste', 'victoria', 'sitp_sabana_larga', 'AB_A31_2', 'sitp_venezuela', 'sitp_nunez', 'sanfelipe', 'AB_A-B_2', 'sitp_ortega', 'AB_A20A_1', 'sitp_tiradentes', 'sitp_cdg_este'}


In [7]:
def dump_links(scenario):
    sm = pool.copy()
    var = parameter_frame[scenario]
    sm.links.loc[sm.links['route_id'].isin(var['route_type'].index), 'route_type'] = 'none'

    for route_id, route_type in var['route_type'].to_dict().items():
        sm.links.loc[sm.links['route_id'] == route_id,  'route_type'] = route_type
        
    sm.links = sm.links.loc[sm.links['route_type'] != 'none']

    for route_type, headway in var['headway'].astype(int).to_dict().items():
        sm.links.loc[sm.links['route_type'] == route_type,  'headway'] = headway

    for route_type, value in var['speed'].astype(float).to_dict().items():
        sm.links.loc[sm.links['route_type'] == route_type,  'speed'] = value
        
    sm.links.dropna(subset=['route_type'], inplace=True)
        
    sm.to_zip(
        quetzal_path + '%s/links.zip' % scenario,
        only_attributes = ['links', 'epsg', 'coordinates_unit']
    )

In [8]:
import os
sm = pool.copy()
for scenario in parameter_frame.columns:
    #print(scenario)
    try:
        os.mkdir(quetzal_path + scenario)
    except FileExistsError:
        pass
    dump_links(scenario)

jsons: 100%|██████████| 2/2 [00:00<00:00,  2.39it/s]
